In [42]:
import os
import random
import numpy as np
from PIL import Image

class RansomNoteGenerator:
    def __init__(self, alphabet_dir):
        """
        Initialize the generator with the directory containing character images.
        """
        self.alphabet_dir = alphabet_dir
        self.char_variants = self._load_character_variants()
        
        # Dimensions (A4 size at 300 DPI)
        self.page_width = 420
        self.page_height = 595
        
        # Margins (10% of width/height)
        self.margin_x = int(self.page_width * 0.05)
        self.margin_y = int(self.page_height * 0.05)
        
    def _load_character_variants(self):
        variants = {}
        for filename in os.listdir(self.alphabet_dir):
            if filename.endswith('.png'):
                char = filename[0]
                if char not in variants:
                    variants[char] = []
                variants[char].append(filename)
        return variants
    
    def _get_random_variant(self, char):
        char = char.upper()
        if char in self.char_variants:
            variant = random.choice(self.char_variants[char])
            img = Image.open(os.path.join(self.alphabet_dir, variant)).convert('RGBA')
            
            # Create a transparent background for the thumbnail
            img.thumbnail((32, 32), Image.Resampling.LANCZOS)  # Resize the image

            # Convert back to PIL Image
            return img
        
        return None

    def _measure_word(self, word, char_spacing):
        """Calculate width and height of a word."""
        width = char_spacing  # Start negative since we'll add one extra spacing
        height = 0
        char_images = []
        
        for char in word:
            img = self._get_random_variant(char)
            if img:
                width += img.size[0] + char_spacing
                height = max(height, img.size[1])
                char_images.append(img)
                
        return width, height, char_images

    def _create_new_page(self, background_color=(254, 248, 241)):
        """Create a new blank page."""
        return Image.new('RGB', (self.page_width, self.page_height), background_color)

    def create_ransom_note(self, text, char_spacing=5, word_spacing=50, line_spacing=20, background_color=(254, 248, 241)):
        """
        Create a ransom note, potentially spanning multiple pages.
        Returns a list of (image, positions) tuples, one for each page.
        """
        words = text.split()
        pages = []  # List to store (image, positions) for each page
        current_page = self._create_new_page(background_color)
        current_positions = []
        
        current_x = self.margin_x
        current_y = self.margin_y
        line_height = 0
        
        def start_new_page():
            nonlocal current_page, current_positions, current_x, current_y, line_height
            # Save current page
            if current_positions:  # Only save if page has content
                pages.append((current_page, current_positions))
            # Start new page
            current_page = self._create_new_page(background_color)
            current_positions = []
            current_x = self.margin_x
            current_y = self.margin_y
            line_height = 0
        
        for word in words:
            # Measure this word
            word_width, word_height, char_images = self._measure_word(word, char_spacing)
            
            # Check if we need to wrap to next line
            if current_x + word_width > self.page_width - self.margin_x and current_x > self.margin_x:
                current_x = self.margin_x
                current_y += line_height + line_spacing
                line_height = 0
            
            # Check if we need to start a new page
            if current_y + word_height > self.page_height - self.margin_y:
                start_new_page()
            
            # Update line height
            line_height = max(line_height, word_height)
            
            # Paste each character of the word
            x = current_x
            for char, img in zip(word, char_images):
                y = current_y + (line_height - img.size[1]) // 2
                current_page.paste(img, (x, y), img)  # Fixed code - using img as its own mask
                current_positions.append((char, x, y, img.size[0], img.size[1]))
                x += img.size[0] + char_spacing
            
            current_x = x + word_spacing - char_spacing
        
        # Add the last page if it has content
        if current_positions:
            pages.append((current_page, current_positions))
        
        return pages

    def create_segmentation_mask(self, char_positions):
        """
        Create a binary segmentation mask that respects character transparency.
        Characters are white (255) and background is black (0).
        """
        mask = np.zeros((self.page_height, self.page_width), dtype=np.uint8)
        
        for char, x, y, w, h in char_positions:
            # Get the character image again to check its alpha channel
            char_img = self._get_random_variant(char)
            if char_img:
                # Convert to numpy array to access alpha channel
                char_array = np.array(char_img)
                # Get alpha channel (1 where visible, 0 where transparent)
                alpha = char_array[..., 3] > 0
                # Only set pixels where the character is visible
                mask[y:y+h, x:x+w][alpha] = 255
        
        return mask

# Example usage:
if __name__ == "__main__":
    generator = RansomNoteGenerator("alphabet")
    
    # Create a multi-page ransom note
    test_text = "irish by a million"
    pages = generator.create_ransom_note(
        test_text,
        char_spacing=-5,
        word_spacing=30,
        line_spacing=50
    )
    
    # Save each page and its corresponding mask
    for i, (page_image, positions) in enumerate(pages):
        # Save the page
        page_image.save(f"test_note_page_{i+1}.png")
        
        # Create and save the segmentation mask for this page
        mask = generator.create_segmentation_mask(positions)
        mask_image = Image.fromarray(mask * 15)  # Scale for visibility
        mask_image.save(f"test_mask_page_{i+1}.png")